In [1]:
#Imports
import numpy as np
import gym

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Dropout
from keras.optimizers import Adam, Adamax, SGD, RMSprop

from rl.agents.dqn import DQNAgent
from rl.policy import BoltzmannQPolicy, EpsGreedyQPolicy, LinearAnnealedPolicy
from rl.memory import SequentialMemory



Using TensorFlow backend.


In [2]:
ENV_NAME = 'LunarLander-v2'
WINDOW_LENGTH = 1 #we only look ahead by 1 state.
record_video_every = 100


env = gym.make(ENV_NAME)
# Todo: Recording video output throwing an error.
# env = gym.wrappers.Monitor(env, 
#                            'recording', 
#                            resume=False,
#                            force=True,
#                            video_callable=None)
nb_actions = env.action_space.n


WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [3]:
#build a model
model = Sequential()
model.add(Flatten(input_shape=(1,) + env.observation_space.shape))
model.add(Dense(32, kernel_initializer='lecun_uniform', activation='relu'))
model.add(Dense(32, kernel_initializer='lecun_uniform', activation='relu'))
model.add(Dense(32, kernel_initializer='lecun_uniform', activation='relu'))
#model.add(LeakyReLU(alpha=0.3))
model.add(Dense(nb_actions))
model.add(Activation('softmax'))

In [4]:
memory = SequentialMemory(
    limit=1000000,                 #Remember previous 1 million states
    window_length=WINDOW_LENGTH)   #Look only 1 state ahead


In [5]:
#train first with eploration, then expoitation.
policy = LinearAnnealedPolicy(EpsGreedyQPolicy(),#train starting with low esp, to high esp.
                              attr='eps',        #ammeal this attr
                              value_max=0.1,       #starting_eps
                              value_min=.001,      #ending:eps
                              value_test=.0001,  #test_mode:eps
                              nb_steps=100000)   #take 1 million steps to slowly alter epsilon.

In [6]:
dqn = DQNAgent(model=model, 
               nb_actions=nb_actions, 
               memory=memory, 
               nb_steps_warmup=1000,     #Run this many before anealing (learn to fall)
               target_model_update=1000, #update model/adjust weights? every 1000 steps?
               policy=policy)

dqn.compile(Adam(lr=.1e-3), metrics=['mse'])

In [11]:
#start training, visualise slows down.
#idsable me in colaboratory
dqn.fit(env, 
        nb_steps=200000, 
        #visualize=True, 
        verbose=1
       )

#save  model after training.
dqn.save_weights('dqn_{}_weights.h5f'.format(ENV_NAME), overwrite=True)

#Finally, evaluate our algorithm for 5 episodes.
dqn.test(env, nb_episodes=200, visualize=True)

Testing for 200 episodes ...
Episode 1: reward: -149.152, steps: 238


KeyboardInterrupt: 

Testing for 200 episodes ...


GLException: b'invalid operation'